# Apartments2Thermal notebook example

In this example, a rule based controller is used to control the model "Apartments2Thermal-v0". At first we import the controller:

In [2]:
import sys
sys.path.insert(0, "/home/lesong/energym/")
from energym.examples.Controller import LabController

Next, we import Energym and create the simulation environment by specifying the model, a weather file and the number of simulation days.

In [3]:
import energym

weather = "ESP_CT_Barcelona"
env = energym.make("Apartments2Thermal-v0", weather=weather, simulation_days=30)

/home/lesong/energym/energym/schedules/EVSchedule.py:60: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  self.schedule["Week"] = self.schedule["Time"].dt.week
/home/lesong/energym/energym/schedules/EVSchedule.py:161: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.schedule.update(df_day)
/home/lesong/energym/energym/schedules/EVSchedule.py:162: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.prediction.update(df_day_pred)
/home/lesong/energym/energym/schedules/EVSch

[OK] fmi2Instantiate: The Resource location of FMU with instance name instance1669805558 is file:///home/lesong/Energym_runs/1669805558_3940059/resources.

[WARNING] fmi2Instantiate: Argument loggingOn is set to 1
. This is not supported. loggingOn will default to '0'.

[OK] The current working directory is /home/lesong/energym/docs/sources/notebooks

[OK] fmi2Instantiate: Path to fmuUnzipLocation ///home/lesong/Energym_runs/1669805558_3940059//

[OK] fmi2Instantiate: Path to fmuResourceLocation ///home/lesong/Energym_runs/1669805558_3940059//resources//

[OK] Command executes to copy content of resources folder: cp -f "///home/lesong/Energym_runs/1669805558_3940059//resources//"variables.cfg "/home/lesong/energym/docs/sources/notebooks//Output_EPExport_instance1669805558//"

[OK] fmi2Instantiate: Path to model description file is ///home/lesong/Energym_runs/1669805558_3940059//modelDescription.xml.

[OK] fmi2Instantiate: The FMU modelIdentifier is Apartments2_heavy_insulated_pump.

[O

/home/lesong/energym/energym/schedules/EVSchedule.py:177: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indices = [self.schedule.index.get_loc(dt, method="nearest") for dt in dts]


Successfully finish reading weather file.
This is the Begin Month: 1
Time (0) set is smaller than minimun allowed (1 day). Day will be set to 1.
This is the Day of the Begin Month: 1
This is the End Month: 1
This is the Day of the End Month: 30
Day of week was left blank in input file.
This is the New Day of Week:  
[OK] This version uses the **energyplus** command line interface to  call the EnergyPlus executable. **RunEPlus.bat** and **runenergyplus** , which were used in earlier versions, were deprecated as of August 2015.
Running EPMacro...
ExpandObjects Started.
No expanded file generated.
ExpandObjects Finished. Time:     0.170
EnergyPlus Starting
EnergyPlus, Version 9.5.0-de239b2e5f, YMD=2022.11.30 10:52


Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]


Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Instantiating FunctionalMockupUnitExport interface
ExternalInterface initializes.
[OK] fmi2EnterInitializationMode: The connection has been accepted.

[OK] fmi2EnterInitializationMode: Slave instance1669805558 is initialized.

Number of outputs in ExternalInterface = 68
Number of inputs  in ExternalInterface = 13
Calculating Detailed Daylighting Factors, Start Date=01/01
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 01/01/2017 for ALL_YEAR
ExternalInterface starts first data exchange.


The control inputs can be inspected using the `get_inputs_names()` method and to construct a controller, we pass the list of inputs and further parameters. This controller determines inputs to get close to the temperature setpoints and uses fixed setpoints during the night.

In [4]:
inputs = env.get_inputs_names()
print(inputs)
controller = LabController(control_list=inputs, lower_tol=0.3, upper_tol=0.8, nighttime_setback=True, nighttime_start=18, nighttime_end=6, nighttime_temp=18)

['Bd_Ch_EV1Bat_sp', 'Bd_Ch_EV2Bat_sp', 'Bd_Pw_Bat_sp', 'P1_T_Thermostat_sp', 'P1_onoff_HP_sp', 'P2_T_Thermostat_sp', 'P2_onoff_HP_sp', 'P3_T_Thermostat_sp', 'P3_onoff_HP_sp', 'P4_T_Thermostat_sp', 'P4_onoff_HP_sp']


In [7]:
env.output_specs

{'Ext_T': {'type': 'scalar',
  'lower_bound': -10,
  'upper_bound': 40,
  'description': 'Outdoor temperature (°C).'},
 'Ext_RH': {'type': 'scalar',
  'lower_bound': 0,
  'upper_bound': 100,
  'description': 'Outdoor relative humidity (%RH).'},
 'Ext_Irr': {'type': 'scalar',
  'lower_bound': 0,
  'upper_bound': 1000,
  'description': 'Direct normal radiation (W/m2).'},
 'Ext_P': {'type': 'scalar',
  'lower_bound': 80000.0,
  'upper_bound': 130000.0,
  'description': 'Outdoor air pressure (Pa).'},
 'P3_T_Thermostat_sp_out': {'type': 'scalar',
  'lower_bound': 16,
  'upper_bound': 26,
  'description': 'Floor 3 thermostat setpoint (°C).'},
 'P4_T_Thermostat_sp_out': {'type': 'scalar',
  'lower_bound': 16,
  'upper_bound': 26,
  'description': 'Floor 4 thermostat setpoint (°C).'},
 'P2_T_Thermostat_sp_out': {'type': 'scalar',
  'lower_bound': 16,
  'upper_bound': 26,
  'description': 'Floor 2 thermostat setpoint (°C).'},
 'P1_T_Thermostat_sp_out': {'type': 'scalar',
  'lower_bound': 16,
  

To run the simulation, a number of steps is specified (here 480 steps per day for 10 days) and the obtained control inputs are passed to the simulation model with the `step()` method. To generate some plots later on, we save all the inputs and outputs in lists.

In [ ]:
steps = 480*10
out_list = []
outputs = env.get_output()
controls = []
hour = 0
for i in range(steps):
    control = controller.get_control(outputs, 21, hour)
    control['Bd_Ch_EV1Bat_sp'] = [0.0]
    control['Bd_Ch_EV2Bat_sp'] = [0.0]
    controls +=[ {p:control[p][0] for p in control} ]
    outputs = env.step(control)
    _,hour,_,_ = env.get_date()
    out_list.append(outputs)

: 

Since the inputs and outputs are given as dictionaries and are collected in lists, we can simply load them as a pandas.DataFrame.

In [ ]:
import pandas as pd
out_df = pd.DataFrame(out_list)

: 

To generate plots, we can directly get the data from the DataFrames, by using the key names. Displayed are the zone temperatures and the setpoints determined by the controller for zone 1 as well as the corresponding heat pump on/off switching, the external temperature, and the total power demand.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

f, (ax1,ax2,ax3) = plt.subplots(3,figsize=(10,15))#


ax1.plot(out_df['Z01_T'], 'r')
ax1.plot(out_df['P1_T_Thermostat_sp_out'], 'b--')
ax1.plot(out_df['P1_onoff_HP_sp_out'], 'orange')
ax1.set_ylabel('Temp')
ax1.set_xlabel('Steps')

ax2.plot(out_df['Ext_T'], 'r')
ax2.set_ylabel('Temp')
ax2.set_xlabel('Steps')

ax3.plot(out_df['Fa_Pw_All'], 'g')
ax3.set_ylabel('Power')
ax3.set_xlabel('Steps')

plt.subplots_adjust(hspace=0.4)

plt.show()

: 

To end the simulation, the `close()` method is called. It deletes files that were produced during the simulation and stores some information about the simulation in the *runs* folder.

In [ ]:
env.close()

: 

: 